In [7]:
import os

In [8]:
%pwd

'f:\\NLP Text Summarizer\\Text-Summarizer\\research'

In [9]:
os.chdir("../")

In [10]:
%pwd

'f:\\NLP Text Summarizer\\Text-Summarizer'

In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [2]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories


In [3]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
         root_dir = config.root_dir,
         source_url = config.source_url,
         local_data_file = config.local_data_file,
         unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [4]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [5]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
               
            
            logger.info(f"{filename} download! with following information: \n{headers}")
        
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
            
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-31 12:49:30,631: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-31 12:49:30,636: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-31 12:49:30,638: INFO: common: created directory at: artifacts]
[2024-01-31 12:49:30,639: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-31 12:49:33,314: INFO: 2237509267: artifacts/data_ingestion/data.zip download! with following information: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6FBE:2AC65A:2FDA93:4876C1:65B9F483
Accept-Ranges: bytes
Date: Wed, 31 Jan 2024 07:19:32 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4739-BOM
X-Ca